# 1. 라이브러리 호출

In [18]:
import pandas as pd
import re
import kss
import krwordrank
from konlpy.tag import Kkma
from konlpy.tag import Okt
#from pykospacing import Spacing
from krwordrank.sentence import summarize_with_sentences
from tqdm import tqdm
import time


kkma = Kkma()
okt = Okt()
#spacing = Spacing()

# 2. 전처리된 블로그 리뷰 불러오기

In [19]:
df = pd.read_csv('processed_place_blog_review3.csv', encoding='utf-8-sig')

In [20]:
reviews = df['blog review']

# 3. 블로그 리뷰 전처리

In [21]:
corpus = []

for i in tqdm(range(0, len(reviews)), desc='전처리 진행율'):
    review = okt.pos(str(reviews[i]), stem=True) # 형태소 분석, 품사태깅
    word = []
    for j in review:
        if not j[1] == 'Noun': # tagging이 명사가 아니면
            continue #건너뛰기
        if len(j[0])==1: # 한 글자짜리는 조사일 수 있으니 건너뛰기
            continue
        word.append(j[0]) # 나머지들을 append
    word = ' '.join(word) # 띄어쓰기로 join
    word += '.' # 끝에 . 붙이기
    corpus.append(word)
    


전처리 진행율: 100%|██████████| 2794/2794 [00:41<00:00, 67.30it/s] 


# 4. 키워드 추출 데이터 추가

In [22]:
df.insert(10, 'blog keyword', corpus) # group by 써서 식당과 묶어 주려고 insert

# 5. 식당과 키워드 연결

In [23]:
df2 = df.groupby('name', sort=False).agg({'blog keyword': ','.join}).reset_index() # 식당 이름으로 이어서 붙이기

In [24]:
keyword = df2['blog keyword']
name = df2['name']

In [25]:
restaurant_keyword = []
for i in keyword:
    restaurant_keyword.append(i.split('.')) # 아까 붙인 . 기준으로 split 해서 이중리스트 형태로 변환

# 6. 최종 키워드 추출

In [26]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
from konlpy.tag import Okt
from PIL import Image
import numpy as np

In [46]:
for i in range(len(name)):
    print(name[i])
    test = restaurant_keyword[i]
    st=''
 
    for j in range(len(test)):
        texts = test[j]
        st+= texts
        texts = st.split('. ')
    try:
        stopwords = {'진짜', '정말', '카페', '테이블', '배곧', '강남', '주문', '모이', '오픈', '추가',
                    '피터', '리조', '직원', '다음', '서비', '가성', '데이', '다른', '점심', '저녁', '오이도'}
        # keywords = KR-WordRank로 학습된 키워드와 랭크 값이 dict형태로, sents = 핵심 문장이 list of str 형식
        keywords, sents = summarize_with_sentences(
                                            texts, #문자열
                                            stopwords = stopwords, #키워드에서 제거할 단어
                                            num_keywords=100, # 키워드로 추출될 키워드 개수
                                            num_keysents=10 # 핵심문장으로 추출될 문장의 개수
                                            )
    except ValueError:
        print('key 없음')
        print()
        continue
        
    for word, j in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:10]: # 키워드 랭킹 순으로 sort
        print('#%s' % word)
    print()

    
    result = sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:20]
    result = dict(result)

    wc = WordCloud(font_path='code/NanumGothic.ttf', width=400, height=400, scale=2.0, max_font_size=250)
    gen = wc.generate_from_frequencies(result)
    wc.to_file(name[i]+'.png')



배곧 또잇카페 배곧점
#스콘
#케이크
#커피
#느낌
#맛집
#공원
#음료
#디저트
#인테리어
#라떼

배곧 카츠오모이
#카츠
#돈까스
#소금
#등심
#안심
#맛집
#카레
#트러플
#사장
#시간

배곧 양푼애등갈비
#갈비
#버섯
#맛집
#양푼
#고기
#양념
#콩나물
#사리
#메밀
#당면

배곧 투파인드피터 배곧점
#파스타
#메뉴
#분위기
#맛집
#크림
#스테이크
#그린
#전복
#매콤
#조명

배곧 소쇄원 배곧본점
#고기
#소쇄원
#맛집
#소고기
#메뉴
#세트
#육회
#부위
#차돌
#미인

배곧 정동진
#조개
#해물
#칼국수
#정동진
#구이
#맛집
#라면
#치즈
#파전
#가리비

배곧 Tailor.Button
#커피
#버튼
#원두
#라떼
#테일러
#작품
#공간
#느낌
#갤러리
#에스

배곧 딱이야직화두마리치킨 배곧점
#치킨
#직화
#마리
#배달
#양념
#맛집
#메뉴
#빠다
#매콤
#치즈

배곧 카페 검은순수 배곧본점
#순수
#맛집
#메뉴
#음료
#라떼
#사장
#크림
#시흥
#아이스
#와플

배곧 Poke all day 포케&샐러드 배곧신도시점
#포케
#샐러드
#소스
#토핑
#맛집
#다이어트
#메밀
#메뉴
#인테리어
#식단

배곧 구십도씨
#도씨
#아이스
#치즈
#메뉴
#커피
#브라운
#음료
#인테리어
#식물
#라떼

배곧 엣홈파스타
#파스타
#뇨끼
#라자냐
#예약
#맛집
#소스
#생면
#메뉴
#오일
#버섯

배곧 세유카츠
#카츠
#돈까스
#돈카츠
#치즈
#맛집
#소금
#소스
#카레
#새우젓
#주산

배곧 모미지
#마카
#미지
#초밥
#참치
#스시
#하나
#전복
#맛집
#식감
#전갱이

배곧 저스트텐동 배곧신도시점
#텐동
#튀김
#저스트
#소스
#메뉴
#맛집
#단호박
#수란
#유자
#베니스

배곧 윌로우 우드
#티라미수
#우드
#로우
#라떼
#분위기
#커피
#인테리어
#느낌
#크런치
#아메리카노

배곧 왈라비스스테이크
#스테이크
#왈라비
#파스타
#메뉴
#분위기
#스프
#샐러드
#와인
#오일
#부라타

배곧 판다소바
#소바
#마제
#다소
#맛집
#식초
#토핑
#라멘